In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, torch, datetime, time
import numpy as np
sys.path.append("..")
import rfi_ml
import matplotlib.pyplot as plt

In [2]:
Ntrain = 100000 #the size of the training set
Ntest = 10
sigma_low = 20
sigma_high = 50
Nepochs = 30
Nrfi = 1
#Np = 2**13 #8k
#hidden_dim = 2**11
Np = 2**11 #2k
hidden_dim = 2**9
z_dim = 16
Nfft = Np // 2 + 1
#Pk = np.ones(Nfft)
Pk = None
#lamb = 0.9 #lambda, scaling down Gaussianized amplitude
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging
start_time = time.time()

t = rfi_ml.ToyGenerator(Np=Np, Pk=Pk)
g = torch.stack([torch.from_numpy(t.getGaussian()) for i in range(Ntrain+Ntest)])
rfi_array = np.zeros((Ntrain+Ntest, Np), dtype=float)
for i in range(Ntrain + Ntest):
    rfi_timestream = np.zeros(Np)
    for j in range(Nrfi):
        rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high)) 
        #rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high), ampl=(0.05, 0.05)) 
        #rfi_timestream += t.getNonGaussianNonlocalized(freq = ((2*np.pi)/(Np), (4*np.pi)/Np), Pflip=0.0) 
    rfi_array[i,:] += rfi_timestream
ng = torch.from_numpy(rfi_array)
#ng = torch.stack([torch.from_numpy(t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high))) for i in range(Ntrain+Ntest)])


Avg irfft(xf):  0.001285004752240652
Avg2 irfft(xf):  0.001285004752240652
Avg irfft(xf):  0.0012666347398003553
Avg2 irfft(xf):  0.0012666347398003553
Avg irfft(xf):  0.0012217854930481264
Avg2 irfft(xf):  0.0012217854930481264
Avg irfft(xf):  0.0002833224803760841
Avg2 irfft(xf):  0.0002833224803760841
Avg irfft(xf):  0.00031969782277403724
Avg2 irfft(xf):  0.00031969782277403724
Avg irfft(xf):  0.0012296296501201087
Avg2 irfft(xf):  0.0012296296501201087
Avg irfft(xf):  0.0013324099375317826
Avg2 irfft(xf):  0.0013324099375317826
Avg irfft(xf):  0.00019868808244154099
Avg2 irfft(xf):  0.00019868808244154099
Avg irfft(xf):  0.0006718593484476703
Avg2 irfft(xf):  0.0006718593484476703
Avg irfft(xf):  0.0010490334577447505
Avg2 irfft(xf):  0.0010490334577447505
Avg irfft(xf):  0.0004380737911949983
Avg2 irfft(xf):  0.0004380737911949983
Avg irfft(xf):  0.0009057820022711621
Avg2 irfft(xf):  0.0009057820022711621
Avg irfft(xf):  0.0005145306493992375
Avg2 irfft(xf):  0.00051453064939923

Avg2 irfft(xf):  0.00019553269287666424
Avg irfft(xf):  0.00048621911854196224
Avg2 irfft(xf):  0.00048621911854196224
Avg irfft(xf):  0.0001964943099446062
Avg2 irfft(xf):  0.0001964943099446062
Avg irfft(xf):  0.00021359516778309402
Avg2 irfft(xf):  0.00021359516778309402
Avg irfft(xf):  4.591403321810143e-05
Avg2 irfft(xf):  4.591403321810143e-05
Avg irfft(xf):  0.00045806999058575234
Avg2 irfft(xf):  0.00045806999058575234
Avg irfft(xf):  0.001306168565622056
Avg2 irfft(xf):  0.001306168565622056
Avg irfft(xf):  0.00018919267266508203
Avg2 irfft(xf):  0.00018919267266508203
Avg irfft(xf):  0.0015334324442770602
Avg2 irfft(xf):  0.0015334324442770602
Avg irfft(xf):  0.00048231129374635794
Avg2 irfft(xf):  0.00048231129374635794
Avg irfft(xf):  0.0002250405155677742
Avg2 irfft(xf):  0.0002250405155677742
Avg irfft(xf):  0.00015360518960236963
Avg2 irfft(xf):  0.00015360518960236963
Avg irfft(xf):  0.000423885596041422
Avg2 irfft(xf):  0.000423885596041422
Avg irfft(xf):  0.0008918694

Avg irfft(xf):  0.00020584912517249344
Avg2 irfft(xf):  0.00020584912517249344
Avg irfft(xf):  0.0004906486249832533
Avg2 irfft(xf):  0.0004906486249832533
Avg irfft(xf):  0.0011483326914406128
Avg2 irfft(xf):  0.0011483326914406128
Avg irfft(xf):  0.00019486172558147994
Avg2 irfft(xf):  0.00019486172558147994
Avg irfft(xf):  0.0009548796822067402
Avg2 irfft(xf):  0.0009548796822067402
Avg irfft(xf):  0.00020839808148895073
Avg2 irfft(xf):  0.00020839808148895073
Avg irfft(xf):  0.0003883034493190929
Avg2 irfft(xf):  0.0003883034493190929
Avg irfft(xf):  0.0002720690412928515
Avg2 irfft(xf):  0.0002720690412928515
Avg irfft(xf):  0.0006338249290885534
Avg2 irfft(xf):  0.0006338249290885534
Avg irfft(xf):  0.0005398050799468642
Avg2 irfft(xf):  0.0005398050799468642
Avg irfft(xf):  0.0007514547452184331
Avg2 irfft(xf):  0.0007514547452184331
Avg irfft(xf):  0.0006726756932087809
Avg2 irfft(xf):  0.0006726756932087809
Avg irfft(xf):  0.00028362224755748105
Avg2 irfft(xf):  0.000283622247

Avg irfft(xf):  0.0005103856544166507
Avg2 irfft(xf):  0.0005103856544166507
Avg irfft(xf):  0.00031819681281837747
Avg2 irfft(xf):  0.00031819681281837747
Avg irfft(xf):  0.0016635182794477199
Avg2 irfft(xf):  0.0016635182794477199
Avg irfft(xf):  0.0003514416236897872
Avg2 irfft(xf):  0.0003514416236897872
Avg irfft(xf):  4.712643260725165e-05
Avg2 irfft(xf):  4.712643260725165e-05
Avg irfft(xf):  0.0008870043405776062
Avg2 irfft(xf):  0.0008870043405776062
Avg irfft(xf):  0.00022600818826719706
Avg2 irfft(xf):  0.00022600818826719706
Avg irfft(xf):  0.00013797435100117172
Avg2 irfft(xf):  0.00013797435100117172
Avg irfft(xf):  0.0004754819633663702
Avg2 irfft(xf):  0.0004754819633663702
Avg irfft(xf):  3.550342367821468e-05
Avg2 irfft(xf):  3.550342367821468e-05
Avg irfft(xf):  0.0005693777068012635
Avg2 irfft(xf):  0.0005693777068012635
Avg irfft(xf):  0.0007476001077114998
Avg2 irfft(xf):  0.0007476001077114998
Avg irfft(xf):  5.715987246164534e-05
Avg2 irfft(xf):  5.7159872461645

Avg irfft(xf):  0.0003449481936904609
Avg2 irfft(xf):  0.0003449481936904609
Avg irfft(xf):  0.0002545679763499635
Avg2 irfft(xf):  0.0002545679763499635
Avg irfft(xf):  8.512225216162532e-05
Avg2 irfft(xf):  8.512225216162532e-05
Avg irfft(xf):  0.0004695296336379787
Avg2 irfft(xf):  0.0004695296336379787
Avg irfft(xf):  0.0006385819649464611
Avg2 irfft(xf):  0.0006385819649464611
Avg irfft(xf):  0.00132305261019126
Avg2 irfft(xf):  0.00132305261019126
Avg irfft(xf):  9.924759938236756e-05
Avg2 irfft(xf):  9.924759938236756e-05
Avg irfft(xf):  0.0006903587238858413
Avg2 irfft(xf):  0.0006903587238858413
Avg irfft(xf):  0.0002707814826108965
Avg2 irfft(xf):  0.0002707814826108965
Avg irfft(xf):  0.00022100511655223323
Avg2 irfft(xf):  0.00022100511655223323
Avg irfft(xf):  0.0011208358918048734
Avg2 irfft(xf):  0.0011208358918048734
Avg irfft(xf):  2.9527527084116266e-06
Avg2 irfft(xf):  2.9527527084116266e-06
Avg irfft(xf):  0.0010057629653522598
Avg2 irfft(xf):  0.0010057629653522598

...

Avg irfft(xf):  0.0006228396048547916
Avg2 irfft(xf):  0.0006228396048547916
Avg irfft(xf):  0.0007232508756862922
Avg2 irfft(xf):  0.0007232508756862922
Avg irfft(xf):  0.0009824385427165845
Avg2 irfft(xf):  0.0009824385427165845
Avg irfft(xf):  0.0003479614373307022
Avg2 irfft(xf):  0.0003479614373307022
Avg irfft(xf):  0.0001992043762957452
Avg2 irfft(xf):  0.0001992043762957452
Avg irfft(xf):  0.00041547317747182437
Avg2 irfft(xf):  0.00041547317747182437
Avg irfft(xf):  0.0006647546624042565
Avg2 irfft(xf):  0.0006647546624042565
Avg irfft(xf):  0.0006166839252090925
Avg2 irfft(xf):  0.0006166839252090925
Avg irfft(xf):  3.7336583516726565e-05
Avg2 irfft(xf):  3.7336583516726565e-05
Avg irfft(xf):  0.0005210554610111003
Avg2 irfft(xf):  0.0005210554610111003
Avg irfft(xf):  0.0008465786210131625
Avg2 irfft(xf):  0.0008465786210131625
Avg irfft(xf):  0.0003044011216290039
Avg2 irfft(xf):  0.0003044011216290039
Avg irfft(xf):  0.00010381014054024413
Avg2 irfft(xf):  0.00010381014054

Avg irfft(xf):  0.000407011409760116
Avg2 irfft(xf):  0.000407011409760116
Avg irfft(xf):  0.0015084364383944964
Avg2 irfft(xf):  0.0015084364383944964
Avg irfft(xf):  0.0005309106953053938
Avg2 irfft(xf):  0.0005309106953053938
Avg irfft(xf):  0.0007848157226283362
Avg2 irfft(xf):  0.0007848157226283362
Avg irfft(xf):  0.00045256149928605497
Avg2 irfft(xf):  0.00045256149928605497
Avg irfft(xf):  0.0006486954328588455
Avg2 irfft(xf):  0.0006486954328588455
Avg irfft(xf):  5.496484622259017e-05
Avg2 irfft(xf):  5.496484622259017e-05
Avg irfft(xf):  5.086510240942093e-05
Avg2 irfft(xf):  5.086510240942093e-05
Avg irfft(xf):  0.0003702919893916854
Avg2 irfft(xf):  0.0003702919893916854
Avg irfft(xf):  0.00020305949267350912
Avg2 irfft(xf):  0.00020305949267350912
Avg irfft(xf):  0.0006216142277173213
Avg2 irfft(xf):  0.0006216142277173213
Avg irfft(xf):  0.0011052651374521315
Avg2 irfft(xf):  0.0011052651374521315
Avg irfft(xf):  9.448746857728264e-05
Avg2 irfft(xf):  9.448746857728264e-

Avg irfft(xf):  0.0006222997771941638
Avg2 irfft(xf):  0.0006222997771941638
Avg irfft(xf):  0.00015452055048899792
Avg2 irfft(xf):  0.00015452055048899792
Avg irfft(xf):  0.0004828212992231606
Avg2 irfft(xf):  0.0004828212992231606
Avg irfft(xf):  0.00041429000021004584
Avg2 irfft(xf):  0.00041429000021004584
Avg irfft(xf):  0.00024030537696960664
Avg2 irfft(xf):  0.00024030537696960664
Avg irfft(xf):  0.00014567881536926684
Avg2 irfft(xf):  0.00014567881536926684
Avg irfft(xf):  1.1834956051093824e-05
Avg2 irfft(xf):  1.1834956051093824e-05
Avg irfft(xf):  0.0001335618473857798
Avg2 irfft(xf):  0.0001335618473857798
Avg irfft(xf):  0.00047850463924718284
Avg2 irfft(xf):  0.00047850463924718284
Avg irfft(xf):  0.0004075840740298091
Avg2 irfft(xf):  0.0004075840740298091
Avg irfft(xf):  0.0006199249029396378
Avg2 irfft(xf):  0.0006199249029396378
Avg irfft(xf):  0.0008826452426850095
Avg2 irfft(xf):  0.0008826452426850095
Avg irfft(xf):  0.0003536083950911399
Avg2 irfft(xf):  0.0003536

Avg irfft(xf):  0.0018814132482168847
Avg2 irfft(xf):  0.0018814132482168847
Avg irfft(xf):  0.0009548256830754424
Avg2 irfft(xf):  0.0009548256830754424
Avg irfft(xf):  0.00030170718256886667
Avg2 irfft(xf):  0.00030170718256886667
Avg irfft(xf):  0.0007817824156780921
Avg2 irfft(xf):  0.0007817824156780921
Avg irfft(xf):  0.001053552580506243
Avg2 irfft(xf):  0.001053552580506243
Avg irfft(xf):  0.0011698331572702737
Avg2 irfft(xf):  0.0011698331572702737
Avg irfft(xf):  0.0005637563807537955
Avg2 irfft(xf):  0.0005637563807537955
Avg irfft(xf):  0.0003246914564301141
Avg2 irfft(xf):  0.0003246914564301141
Avg irfft(xf):  0.0004523395143973782
Avg2 irfft(xf):  0.0004523395143973782
Avg irfft(xf):  0.00037584150291546407
Avg2 irfft(xf):  0.00037584150291546407
Avg irfft(xf):  0.00017481608028552373
Avg2 irfft(xf):  0.00017481608028552373
Avg irfft(xf):  0.002024146400312927
Avg2 irfft(xf):  0.002024146400312927
Avg irfft(xf):  0.0005775484586103912
Avg2 irfft(xf):  0.00057754845861039

KeyboardInterrupt: 

In [ ]:
n = rfi_ml.RFIDetect(Np, Nepochs = Nepochs, z_dim = z_dim, hidden_dim = hidden_dim)

In [ ]:
n.train(g[:Ntrain], ng[:Ntrain])

In [ ]:
start_test = time.time()
testout = n.evaluate(g[Ntrain:], ng[Ntrain:])
end_test = time.time()

In [ ]:
n.plot_eval(testout, g[Ntrain:], ng[Ntrain:])

In [ ]:
print("Runtime: ", time.time()-start_time)
print("Evaltime: ", end_test-start_test)